# Notebook for Text Analysis
## Author: Sanittawan Tan
## Date: March 18, 2020

- Conduct topic modeling on IO posts

## Install required libraies

In [0]:
!pip install -U git+git://github.com/Computational-Content-Analysis-2020/lucem_illud_2020.git

  Cloning git://github.com/Computational-Content-Analysis-2020/lucem_illud_2020.git to /tmp/pip-req-build-676u_ik5
  Running command git clone -q git://github.com/Computational-Content-Analysis-2020/lucem_illud_2020.git /tmp/pip-req-build-676u_ik5
  Created wheel for lucem-illud-2020: filename=lucem_illud_2020-8.0.1-cp36-none-any.whl size=35151 sha256=c0f800d5bdda3bffb3c65ce982d019685541a5d65a3347a2855cd9b282f80993
  Stored in directory: /tmp/pip-ephem-wheel-cache-o_s8i_za/wheels/a8/16/91/3c63788e494d360378317fe5ec9f4972f661844af8ae8c26f0
Successfully built lucem-illud-2020
  Found existing installation: lucem-illud-2020 8.0.1
    Uninstalling lucem-illud-2020-8.0.1:
      Successfully uninstalled lucem-illud-2020-8.0.1


In [0]:
!pip install --upgrade gensim

     |████████████████████████████████| 24.2MB 162kB/s 
  Found existing installation: gensim 3.6.0
    Uninstalling gensim-3.6.0:
      Successfully uninstalled gensim-3.6.0


In [0]:
!pip install deepcut

     |████████████████████████████████| 2.0MB 2.8MB/s 
     |████████████████████████████████| 421.8MB 37kB/s 
     |████████████████████████████████| 450kB 42.4MB/s 
     |████████████████████████████████| 3.9MB 51.9MB/s 
ERROR: tensorflow-federated 0.12.0 has requirement tensorflow-addons~=0.7.0, but you'll have tensorflow-addons 0.8.3 which is incompatible.
  Found existing installation: tensorflow-estimator 1.15.1
    Uninstalling tensorflow-estimator-1.15.1:
      Successfully uninstalled tensorflow-estimator-1.15.1
  Found existing installation: tensorboard 1.15.0
    Uninstalling tensorboard-1.15.0:
      Successfully uninstalled tensorboard-1.15.0
  Found existing installation: tensorflow 1.15.0
    Uninstalling tensorflow-1.15.0:
      Successfully uninstalled tensorflow-1.15.0


In [0]:
!pip install pythainlp

     |████████████████████████████████| 11.1MB 2.7MB/s 
     |████████████████████████████████| 1.5MB 44.3MB/s 
     |████████████████████████████████| 61kB 7.4MB/s 
  Created wheel for nltk: filename=nltk-3.4.5-cp36-none-any.whl size=1449905 sha256=b45739b6a994ffaab379700fdc927188baa75ffa9f5b9d33649049da81c22490
  Stored in directory: /root/.cache/pip/wheels/96/86/f6/68ab24c23f207c0077381a5e3904b2815136b879538a24b483
Successfully built nltk
ERROR: google-colab 1.0.0 has requirement requests~=2.21.0, but you'll have requests 2.23.0 which is incompatible.
ERROR: datascience 0.10.6 has requirement folium==0.2.1, but you'll have folium 0.8.3 which is incompatible.
  Found existing installation: nltk 3.2.5
    Uninstalling nltk-3.2.5:
      Successfully uninstalled nltk-3.2.5
  Found existing installation: requests 2.21.0
    Uninstalling requests-2.21.0:
      Successfully uninstalled requests-2.21.0


In [0]:
!pip install emoji

     |████████████████████████████████| 51kB 1.7MB/s 
  Created wheel for emoji: filename=emoji-0.5.4-cp36-none-any.whl size=42176 sha256=dcfe2373196ebb8b250152182f8cf432bbb036322d318af566ee9b3da9feab30
  Stored in directory: /root/.cache/pip/wheels/2a/a9/0a/4f8e8cce8074232aba240caca3fade315bb49fac68808d1a9c
Successfully built emoji


In [0]:
import pandas as pd
import numpy as np
import sklearn
import matplotlib.pyplot as plt
import matplotlib.cm 
import seaborn as sns
import lucem_illud_2020
import gensim
import deepcut

## Load and Prepare Data

In [0]:
io_df = pd.read_csv('predicted_io.csv', encoding='utf-8')

In [0]:
io_df.head(2)

,screen_name,fullname,text,timestamp,following,followers,num_tweets,location,num_likes,id,date_joined,num_days_joined,days_on_twitter,IO_meta_clf,isIO_meta_prob,is_IO,is_IO_prob
0,KI36bsxg8ozf6bg,ชัย สุขกายสบายใจ,ดูข่าวให้ดี อย่าเชื่อข่าวปลอม แล้วค่อยมาแสดงึว...,2020-02-02 17:52:19,36,22,2673,-1,0,1201755584225107968,2019-12-02 22:50:00,86 days 01:10:00.000000000,86,1,0.972172,1,0.937824
1,thaiprong,รวมใจไทยปรองดอง,ข่าวดี!! แพทย์ราชวิถี รักษาผู้ป่วยโคโรนา อากา...,2020-02-02 15:15:13,304,99,78,-1,9,1209024400651014144,2019-12-23 00:14:00,65 days 23:46:00.000000000,65,1,0.916954,1,0.917706


In [0]:
# try cleaning the text
# code from https://colab.research.google.com/drive/1uaOsotWWgHGGceXkoE75xzSdlYsMZ4RK#scrollTo=LOcncIVCkvw6
import re
import string

def clean_msg(msg):
  
    # ลบ url ทั้งหมด
    msg = re.sub(r'(https?://\S+)','', msg)
    msg = re.sub(r'pic\.twitter\.com\/\S+', '', msg)

    # delete zero width space '\u200b'
    msg = msg.replace('\u200b', '')

    # ลบ horizontal ellipsis and quotation marks
    msg = re.sub(r'[…]','', msg)
    msg = re.sub(r'[”]','', msg)
    msg = re.sub(r'[“]','', msg)
    msg = re.sub(r'[฿]','', msg)
    msg = re.sub(r'[•]','', msg)

    # ลบ text ที่อยู่ในวงเล็บ <> ทั้งหมด
    msg = re.sub(r'<.*?>','', msg)
    
    # ลบ hashtag
    msg = re.sub(r'#','',msg)
    
    # ลบ เครื่องหมายคำพูด (punctuation)
    for c in string.punctuation:
        msg = re.sub(r'\{}'.format(c),'',msg)
    
    # ลบ separator เช่น \n \t
    msg = ' '.join(msg.split())
    
    return msg

In [0]:
print(io_df.iloc[1]['text'])
print(clean_msg(io_df.iloc[1]['text']))

 ข่าวดี!! แพทย์ราชวิถี รักษาผู้ป่วยโคโรนา อาการดีขึ้นภายใน 48 ชั่วโมง
#ไวรัสโคโรนา
#ไวรัสอู่ฮั่น
#รัฐบาลเตรียมพร้อม
#เสพสื่ออย่างมีสติ
#อย่าตื่นตระหนก
#Fakenews
#ข่าวปลอม
https://www.facebook.com/thaiprong2019/?modal=admin_todo_tour …pic.twitter.com/Pw1pg2ABZf
ข่าวดี แพทย์ราชวิถี รักษาผู้ป่วยโคโรนา อาการดีขึ้นภายใน 48 ชั่วโมง ไวรัสโคโรนา ไวรัสอู่ฮั่น รัฐบาลเตรียมพร้อม เสพสื่ออย่างมีสติ อย่าตื่นตระหนก Fakenews ข่าวปลอม


In [0]:
io_df['cleaned_text'] = io_df['text'].apply(lambda x: clean_msg(x))

In [0]:
io_df['cleaned_text'].values

array(['ดูข่าวให้ดี อย่าเชื่อข่าวปลอม แล้วค่อยมาแสดงึวาทคิดเห็นนะครับ รัฐบาลเตรียมพร้อม',
       'ข่าวดี แพทย์ราชวิถี รักษาผู้ป่วยโคโรนา อาการดีขึ้นภายใน 48 ชั่วโมง ไวรัสโคโรนา ไวรัสอู่ฮั่น รัฐบาลเตรียมพร้อม เสพสื่ออย่างมีสติ อย่าตื่นตระหนก Fakenews ข่าวปลอม',
       'ดูข่าวให้ดี อย่าเชื่อข่าวปลอม แล้วค่อยมาแสดงึวาทคิดเห็นนะครับ รัฐบาลเตรียมพร้อม',
       'ข่าวดี แพทย์ราชวิถี รักษาผู้ป่วยโคโรนา อาการดีขึ้นภายใน 48 ชั่วโมง ไวรัสโคโรนา ไวรัสอู่ฮั่น รัฐบาลเตรียมพร้อม เสพสื่ออย่างมีสติ อย่าตื่นตระหนก Fakenews ข่าวปลอม',
       'อัปเดตไวรัส โคโรนา ประจำวันที่ 1 กพ 63 ในไทย ไวรัสโคโรนา ไวรัสอู่ฮั่น รัฐบาลเตรียมพร้อม เสพสื่ออย่างมีสติ อย่าตื่นตระหนก Fakenews ข่าวปลอม',
       'ต้องบอกว่ารัฐพยายามประชาสัมพันธ์ให้ผู้คนเกิดความตื่นตัวในการป้องกันไวรัส ขอบคุณกองทัพบกที่พยายามประชาสัมพันธ์ให้คนไทยได้รับรู้มากขึ้น ไวรัสโคโรนา ไวรัสอู่ฮั่น รัฐบาลเตรียมพร้อม เสพสื่ออย่างมีสติ อย่าตื่นตระหนก',
       'กระผมหนึ่งในพลเมืองคนไทย ขอเป็นกำลังใจให้คุณหมอ และทีมงานทุกๆท่านที่ทำหน้าอยู่ ณ จุดแสกนคัดกรอง ขอส่ง

### Tokenize and Normalize Text

#### Try customizing the tf-idf

In [0]:
from pythainlp.ulmfit import process_thai
from pythainlp.util import normalize

Corpus: wiki_lm_lstm
- Downloading: wiki_lm_lstm 0.32


100%|██████████| 1050919089/1050919089 [00:29<00:00, 35728033.77it/s]


Corpus: wiki_itos_lstm
- Downloading: wiki_itos_lstm 0.32


100%|██████████| 1530484/1530484 [00:00<00:00, 13113756.95it/s]


In [0]:
# clean the text
def normalize_words(tokenized_textlist):
  normal = []
  for word in tokenized_textlist:
    if word in ['#', ' ', 'ๆ']:
      continue
    word = word.strip()
    norm_word = normalize(word)
    normal.append(norm_word)
  return normal

In [0]:
io_df['tokenized_text'] = io_df['cleaned_text'].apply(lambda x: deepcut.tokenize(x))
io_df['normalized_text'] = io_df['tokenized_text'].apply(lambda x: normalize_words(x))

In [0]:
io_df.iloc[0]['normalized_text']

['ดู',
 'ข่าว',
 'ให้',
 'ดี',
 'อย่า',
 'เชื่อ',
 'ข่าว',
 'ปลอม',
 'แล้ว',
 'ค่อย',
 'มา',
 'แสด',
 'งึวาท',
 'คิดเห็น',
 'นะ',
 'ครับ',
 'รัฐบาล',
 'เตรียม',
 'พร้อม']

In [0]:
tokens_list_j = [','.join(tkn) for tkn in io_df['normalized_text'].values]

In [0]:
from sklearn.feature_extraction.text import CountVectorizer

cvec = CountVectorizer(analyzer=lambda x: x.split(','))
c_feat = cvec.fit_transform(tokens_list_j)

In [0]:
cvec.vocabulary_

{'ดู': 302,
 'ข่าว': 146,
 'ให้': 1123,
 'ดี': 299,
 'อย่า': 831,
 'เชื่อ': 908,
 'ปลอม': 475,
 'แล้ว': 1068,
 'ค่อย': 195,
 'มา': 571,
 'แสด': 1073,
 'งึวาท': 211,
 'คิดเห็น': 183,
 'นะ': 395,
 'ครับ': 162,
 'รัฐบาล': 626,
 'เตรียม': 924,
 'พร้อม': 525,
 'แพทย์': 1054,
 'ราชวิถี': 637,
 'รักษา': 623,
 'ผู้': 507,
 'ป่วย': 490,
 'โคโรนา': 1088,
 'อาการ': 845,
 'ขึ้น': 142,
 'ภาย': 555,
 'ใน': 1118,
 '48': 17,
 'ชั่วโมง': 259,
 'ไวรัส': 1135,
 'อู่ฮั่น': 867,
 'เสพ': 990,
 'สื่อ': 756,
 'อย่าง': 832,
 'มี': 581,
 'สติ': 713,
 'ตื่นตระหนก': 335,
 'Fakenews': 32,
 'อัปเดตไวรัส': 844,
 'ประจำ': 453,
 'วัน': 686,
 'ที่': 385,
 '1': 0,
 'กพ': 77,
 '63': 21,
 'ไทย': 1127,
 'ต้อง': 344,
 'บอก': 425,
 'ว่า': 703,
 'รัฐ': 624,
 'พยายาม': 519,
 'ประชาสัมพันธ์': 456,
 'ผู้คน': 508,
 'เกิด': 879,
 'ความ': 172,
 'ตื่นตัว': 336,
 'การ': 109,
 'ป้องกัน': 493,
 'ขอบคุณ': 133,
 'กองทัพบก': 99,
 'คน': 158,
 'ได้': 1126,
 'รับ': 631,
 'รู้': 646,
 'มาก': 572,
 'กระผม': 84,
 'หนึ่ง': 776,
 'พลเมือง': 528,


#### Use Sklearn Tf-idf vectorizer

In [0]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf = TfidfVectorizer(tokenizer=process_thai, ngram_range=(1,1), min_df=2, sublinear_tf=True)
TFVects = tfidf.fit_transform(io_df['cleaned_text'])

In [0]:
tfidf.vocabulary_

{'  ': 0,
 '1': 1,
 '10': 2,
 '11': 3,
 '112': 4,
 '13': 5,
 '19': 6,
 '2': 7,
 '2019': 8,
 '2020': 9,
 '3': 10,
 '4': 11,
 '48': 12,
 '5': 13,
 '63': 14,
 'coronavirus': 15,
 'covid': 16,
 'crystalmoment': 17,
 'fake': 18,
 'fakenews': 19,
 'happyvalentinesday': 20,
 'io': 21,
 'iphone': 22,
 'keepwalking': 23,
 'linetoday': 24,
 'save': 25,
 'saveg': 26,
 'seabournovation': 27,
 'valentinesday': 28,
 'westerdam': 29,
 'xxrep': 30,
 'xxwrep': 31,
 'กน': 32,
 'กพ': 33,
 'กรมการแพทย์': 34,
 'กรอง': 35,
 'กระตุก': 36,
 'กระผม': 37,
 'กราด': 38,
 'กรุง': 39,
 'กลับบ้าน': 40,
 'กลับมา': 41,
 'กลัว': 42,
 'กลั้น': 43,
 'กลางกรุง': 44,
 'กล้า': 45,
 'กว่า': 46,
 'กองทัพ': 47,
 'กองทัพบก': 48,
 'กองปราบ': 49,
 'กะ': 50,
 'กัน': 51,
 'กับ': 52,
 'การ': 53,
 'การป้องกัน': 54,
 'การพัฒนา': 55,
 'การรักษา': 56,
 'การแก้ปัญหา': 57,
 'กำจัด': 58,
 'กำลัง': 59,
 'กำลังใจ': 60,
 'กิน': 61,
 'กู': 62,
 'ก็': 63,
 'ก็ดี': 64,
 'ก่อน': 65,
 'ขณะนี้': 66,
 'ขน': 67,
 'ขอ': 68,
 'ของ': 69,
 'ขอบคุณ': 70,


In [0]:
bigram_tfidf = TfidfVectorizer(tokenizer=process_thai, ngram_range=(2,2), min_df=2, sublinear_tf=True)
bigram_TFVects = bigram_tfidf.fit_transform(io_df['cleaned_text'])
bigram_tfidf.vocabulary_

{'ดู ข่าว': 283,
 'ข่าว ให้': 121,
 'ให้ ดี': 1119,
 'ดี อย่า': 274,
 'อย่า เชื่อ': 828,
 'เชื่อ ข่าว': 895,
 'ข่าว ปลอม': 118,
 'ปลอม แล้ว': 523,
 'แล้ว ค่อย': 1050,
 'ค่อย มา': 197,
 'มา แสด': 608,
 'แสด งึ': 1060,
 'งึ วาท': 208,
 'วาท คิดเห็น': 711,
 'คิดเห็น นะ': 186,
 'นะ ครับ': 445,
 'ครับ รัฐบาล': 155,
 'รัฐบาล เตรียมพร้อม': 675,
 'ข่าวดี แพทย์': 122,
 'แพทย์ ราชวิถี': 1024,
 'ราชวิถี รักษา': 688,
 'รักษา ผู้ป่วย': 658,
 'ผู้ป่วย โค': 551,
 'โค โร': 1065,
 'โร นา': 1081,
 'นา อาการ': 460,
 'อาการ ดีขึ้น': 848,
 'ดีขึ้น ภายใน': 279,
 'ภายใน 48': 588,
 '48 ชั่วโมง': 14,
 'ชั่วโมง ไวรัส': 241,
 'ไวรัส โค': 1190,
 'นา ไวรัส': 461,
 'ไวรัส อู่': 1189,
 'อู่ ฮั่น': 858,
 'ฮั่น รัฐบาล': 862,
 'เตรียมพร้อม เสพ': 918,
 'เสพ สื่อ': 979,
 'สื่อ อย่าง': 757,
 'อย่าง มีสติ': 831,
 'มีสติ อย่า': 626,
 'อย่า ตื่นตระหนก': 824,
 'ตื่นตระหนก fakenews': 323,
 'fakenews ข่าว': 19,
 'อัปเดต ไวรัส': 847,
 'นา ประจำ': 457,
 'ประจำ วันที่': 500,
 'วันที่ 1': 706,
 '1 กพ': 1,
 'กพ 63': 29,
 '63 ใน': 16

In [0]:
bigram_tfidf.vocabulary_

{'ดู ข่าว': 283,
 'ข่าว ให้': 121,
 'ให้ ดี': 1119,
 'ดี อย่า': 274,
 'อย่า เชื่อ': 828,
 'เชื่อ ข่าว': 895,
 'ข่าว ปลอม': 118,
 'ปลอม แล้ว': 523,
 'แล้ว ค่อย': 1050,
 'ค่อย มา': 197,
 'มา แสด': 608,
 'แสด งึ': 1060,
 'งึ วาท': 208,
 'วาท คิดเห็น': 711,
 'คิดเห็น นะ': 186,
 'นะ ครับ': 445,
 'ครับ รัฐบาล': 155,
 'รัฐบาล เตรียมพร้อม': 675,
 'ข่าวดี แพทย์': 122,
 'แพทย์ ราชวิถี': 1024,
 'ราชวิถี รักษา': 688,
 'รักษา ผู้ป่วย': 658,
 'ผู้ป่วย โค': 551,
 'โค โร': 1065,
 'โร นา': 1081,
 'นา อาการ': 460,
 'อาการ ดีขึ้น': 848,
 'ดีขึ้น ภายใน': 279,
 'ภายใน 48': 588,
 '48 ชั่วโมง': 14,
 'ชั่วโมง ไวรัส': 241,
 'ไวรัส โค': 1190,
 'นา ไวรัส': 461,
 'ไวรัส อู่': 1189,
 'อู่ ฮั่น': 858,
 'ฮั่น รัฐบาล': 862,
 'เตรียมพร้อม เสพ': 918,
 'เสพ สื่อ': 979,
 'สื่อ อย่าง': 757,
 'อย่าง มีสติ': 831,
 'มีสติ อย่า': 626,
 'อย่า ตื่นตระหนก': 824,
 'ตื่นตระหนก fakenews': 323,
 'fakenews ข่าว': 19,
 'อัปเดต ไวรัส': 847,
 'นา ประจำ': 457,
 'ประจำ วันที่': 500,
 'วันที่ 1': 706,
 '1 กพ': 1,
 'กพ 63': 29,
 '63 ใน': 16

#### make corpus (unigram)

In [0]:
def dropMissing(wordLst, vocab):
    return [w for w in wordLst if w in vocab]

In [0]:
# use cvec here, but can also use tfidf
io_df['reduced_tokens'] = io_df['normalized_text'].apply(
    lambda x: dropMissing(x, cvec.vocabulary_.keys()))

In [0]:
io_df['reduced_tokens']

0      [ดู, ข่าว, ให้, ดี, อย่า, เชื่อ, ข่าว, ปลอม, แ...
1      [ข่าว, ดี, แพทย์, ราชวิถี, รักษา, ผู้, ป่วย, โ...
2      [ดู, ข่าว, ให้, ดี, อย่า, เชื่อ, ข่าว, ปลอม, แ...
3      [ข่าว, ดี, แพทย์, ราชวิถี, รักษา, ผู้, ป่วย, โ...
4      [อัปเดตไวรัส, โคโรนา, ประจำ, วัน, ที่, 1, กพ, ...
                             ...                        
370    [เตรียม, พร้อม, เหี้ย, ไร, หละ, อยาก, จะ, IO, ...
371    [รัฐบาล, เตรียม, พร้อม, ผนงรจตกม, Terminal, 21...
372    [รัฐบาล, เตรียม, พร้อม, ทั้ง, สี, ข้าง, และ, 8...
373    [รัฐบาล, ก็, เห็น, ขยัน, ดี, นะ, ขยัน, ทำ, ให้...
374    [เอา, เงิน, ค่า, จ้าง, IO, มา, พัฒนา, ระบบ, กา...
Name: reduced_tokens, Length: 375, dtype: object

In [0]:
dictionary = gensim.corpora.Dictionary(io_df['reduced_tokens'])

In [0]:
corpus = [dictionary.doc2bow(text) for text in io_df['reduced_tokens']]

In [0]:
# serialize the corpus and load it
gensim.corpora.MmCorpus.serialize('io.mm', corpus)
iomm = gensim.corpora.MmCorpus('io.mm')

#### make corpus (bigram)

In [0]:
analyzer = bigram_tfidf.build_analyzer()
print(analyzer(io_df.iloc[0]['cleaned_text']))

io_df['bigram_tokens'] = io_df['cleaned_text'].apply(
    lambda x: analyzer(x))

['ดู ข่าว', 'ข่าว ให้', 'ให้ ดี', 'ดี อย่า', 'อย่า เชื่อ', 'เชื่อ ข่าว', 'ข่าว ปลอม', 'ปลอม แล้ว', 'แล้ว ค่อย', 'ค่อย มา', 'มา แสด', 'แสด งึ', 'งึ วาท', 'วาท คิดเห็น', 'คิดเห็น นะ', 'นะ ครับ', 'ครับ รัฐบาล', 'รัฐบาล เตรียมพร้อม']


In [0]:
io_df['bigram_tokens'].values[:5]

array([list(['ดู ข่าว', 'ข่าว ให้', 'ให้ ดี', 'ดี อย่า', 'อย่า เชื่อ', 'เชื่อ ข่าว', 'ข่าว ปลอม', 'ปลอม แล้ว', 'แล้ว ค่อย', 'ค่อย มา', 'มา แสด', 'แสด งึ', 'งึ วาท', 'วาท คิดเห็น', 'คิดเห็น นะ', 'นะ ครับ', 'ครับ รัฐบาล', 'รัฐบาล เตรียมพร้อม']),
       list(['ข่าวดี แพทย์', 'แพทย์ ราชวิถี', 'ราชวิถี รักษา', 'รักษา ผู้ป่วย', 'ผู้ป่วย โค', 'โค โร', 'โร นา', 'นา อาการ', 'อาการ ดีขึ้น', 'ดีขึ้น ภายใน', 'ภายใน 48', '48 ชั่วโมง', 'ชั่วโมง ไวรัส', 'ไวรัส โค', 'โค โร', 'โร นา', 'นา ไวรัส', 'ไวรัส อู่', 'อู่ ฮั่น', 'ฮั่น รัฐบาล', 'รัฐบาล เตรียมพร้อม', 'เตรียมพร้อม เสพ', 'เสพ สื่อ', 'สื่อ อย่าง', 'อย่าง มีสติ', 'มีสติ อย่า', 'อย่า ตื่นตระหนก', 'ตื่นตระหนก fakenews', 'fakenews ข่าว', 'ข่าว ปลอม']),
       list(['ดู ข่าว', 'ข่าว ให้', 'ให้ ดี', 'ดี อย่า', 'อย่า เชื่อ', 'เชื่อ ข่าว', 'ข่าว ปลอม', 'ปลอม แล้ว', 'แล้ว ค่อย', 'ค่อย มา', 'มา แสด', 'แสด งึ', 'งึ วาท', 'วาท คิดเห็น', 'คิดเห็น นะ', 'นะ ครับ', 'ครับ รัฐบาล', 'รัฐบาล เตรียมพร้อม']),
       list(['ข่าวดี แพทย์', 'แพทย์ ราชวิถี', 'ราชวิถี รักษา'

In [0]:
io_df['reduced_bitokens'] = io_df['bigram_tokens'].apply(
    lambda x: dropMissing(x, bigram_tfidf.vocabulary_.keys()))

In [0]:
io_df.iloc[:5]['reduced_bitokens']

0    [ดู ข่าว, ข่าว ให้, ให้ ดี, ดี อย่า, อย่า เชื่...
1    [ข่าวดี แพทย์, แพทย์ ราชวิถี, ราชวิถี รักษา, ร...
2    [ดู ข่าว, ข่าว ให้, ให้ ดี, ดี อย่า, อย่า เชื่...
3    [ข่าวดี แพทย์, แพทย์ ราชวิถี, ราชวิถี รักษา, ร...
4    [อัปเดต ไวรัส, ไวรัส โค, โค โร, โร นา, นา ประจ...
Name: reduced_bitokens, dtype: object

In [0]:
dictionary_bg = gensim.corpora.Dictionary(io_df['reduced_bitokens'])

In [0]:
corpus_bg = [dictionary_bg.doc2bow(text) for text in io_df['reduced_bitokens']]

In [0]:
# serialize the corpus and load it
gensim.corpora.MmCorpus.serialize('io_bg.mm', corpus_bg)
iomm_bg = gensim.corpora.MmCorpus('io_bg.mm')

## Topic Modeling

### 5 topics

In [0]:
# 5 topics
iolda5 = gensim.models.ldamodel.LdaModel(corpus=iomm, id2word=dictionary, 
                                        num_topics=5, alpha='auto', eta='auto')

In [0]:
iolda5.show_topic(1)

[('ไวรัส', 0.037416413),
 ('เตรียม', 0.027691163),
 ('รัฐบาล', 0.026396528),
 ('พร้อม', 0.025906838),
 ('ให้', 0.024230517),
 ('มี', 0.023194533),
 ('ท่าน', 0.02216276),
 ('ทุก', 0.02106485),
 ('โคโรนา', 0.019970836),
 ('ขอ', 0.019664308)]

In [0]:
print(iolda5.num_topics)
print(iolda5.num_terms)

5
1166


In [0]:
def create_topics_df(lda_model, num_words):
  topicsDict = {}
  for topicNum in range(lda_model.num_topics):
      topicWords = [w for w, p in lda_model.show_topic(topicNum,  
                                                       topn=num_words)]
      topicsDict['Topic_{}'.format(topicNum)] = topicWords

  wordRanksDF = pd.DataFrame(topicsDict)
  return wordRanksDF

In [0]:
topics5_untuned = create_topics_df(iolda5, 20)
topics5_untuned

,Topic_0,Topic_1,Topic_2,Topic_3,Topic_4
0,รัฐบาล,ไวรัส,เตรียม,รัฐบาล,พร้อม
1,เตรียม,เตรียม,พร้อม,พร้อม,รัฐบาล
2,พร้อม,รัฐบาล,รัฐบาล,เตรียม,เตรียม
3,ไม่,พร้อม,ไวรัส,ให้,ไวรัส
4,คน,ให้,ไม่,ได้,ให้
5,ดี,มี,อย่าง,มี,มี
6,กัน,ท่าน,ที่,ที่,ที่
7,ให้,ทุก,มี,ครับ,คน
8,มา,โคโรนา,ข่าว,ข่าว,อย่าง
9,ที่,ขอ,สื่อ,คน,สื่อ


In [0]:
# try tuning 
iolda5_tuned = gensim.models.ldamodel.LdaModel(corpus=iomm, id2word=dictionary, 
                                        num_topics=5, alpha='asymmetric', eta=0.9)

topics5_tuned = create_topics_df(iolda5_tuned, 20)
topics5_tuned

,Topic_0,Topic_1,Topic_2,Topic_3,Topic_4
0,รัฐบาล,เตรียม,พร้อม,ข่าว,ไวรัส
1,พร้อม,พร้อม,รัฐบาล,รัฐบาล,พร้อม
2,เตรียม,รัฐบาล,เตรียม,ครับ,เตรียม
3,ไวรัส,ไวรัส,ให้,ปลอม,รัฐบาล
4,มี,ข่าว,ไวรัส,พร้อม,ให้
5,ให้,ที่,ท่าน,เตรียม,ไม่
6,อย่า,ได้,ทุก,ได้,ข่าว
7,อย่าง,ครับ,ที่,ที่,ว่า
8,ไม่,มี,มี,ไวรัส,ได้
9,ครับ,ปลอม,คน,การ,มี


In [0]:
topics5_tuned.to_csv("unigram_tm.csv", index=True, encoding='utf-8')


### 5 topics with bigram

In [0]:
# 5 topics
iolda_bg5 = gensim.models.ldamodel.LdaModel(corpus=iomm_bg, id2word=dictionary_bg, 
                                        num_topics=5, alpha='auto', eta='auto')

In [0]:
topics5bg_untuned = create_topics_df(iolda_bg5, 20)
topics5bg_untuned

,Topic_0,Topic_1,Topic_2,Topic_3,Topic_4
0,รัฐบาล เตรียมพร้อม,รัฐบาล เตรียมพร้อม,รัฐบาล เตรียมพร้อม,รัฐบาล เตรียมพร้อม,รัฐบาล เตรียมพร้อม
1,สื่อ อย่าง,โค โร,พวก ท่าน,ไวรัส โค,เสพ สื่อ
2,อย่า ตื่นตระหนก,ไวรัส โค,กำลังใจ ให้,ข่าว ปลอม,โร นา
3,โค โร,สายพันธุ์ ใหม่,ไวรัส โค,ฮั่น รัฐบาล,กำลังใจ ให้
4,อู่ ฮั่น,โร นา,โค โร,โร นา,สื่อ อย่าง
5,เตรียมพร้อม เสพ,โร น่า,มีสติ อย่า,อย่าง มีสติ,พวก ท่าน
6,มีสติ อย่า,ใหม่ 2019,อย่าง มีสติ,ไวรัส อู่,เตรียมพร้อม เสพ
7,เสพ สื่อ,น่า สายพันธุ์,ไวรัส อู่,เตรียมพร้อม เสพ,โค โร
8,โร นา,เสพ สื่อ,สื่อ อย่าง,โค โร,ไวรัส โค
9,อย่าง มีสติ,ไวรัส อู่,อู่ ฮั่น,อย่า ตื่นตระหนก,อย่า ตื่นตระหนก


In [0]:
topics5bg_untuned.to_csv("bigram_tm.csv", index=True, encoding='utf-8')

## Explore repetitive posts and users

In [0]:
io_df.columns

Index(['screen_name', 'fullname', 'text', 'timestamp', 'following',
       'followers', 'num_tweets', 'location', 'num_likes', 'id', 'date_joined',
       'num_days_joined', 'days_on_twitter', 'IO_meta_clf', 'isIO_meta_prob',
       'is_IO', 'is_IO_prob', 'cleaned_text', 'tokenized_text',
       'normalized_text', 'reduced_tokens', 'bigram_tokens',
       'reduced_bitokens'],
      dtype='object')

In [0]:
io_df.iloc[50:100][['screen_name', 'fullname', 'cleaned_text', 'timestamp', 'is_IO']]

,screen_name,fullname,cleaned_text,timestamp,is_IO
50,thaiprong,รวมใจไทยปรองดอง,อีกด้านหนึ่งของสังคมไทย ที่ได้มองเห็นแล้วก็รู้...,2020-01-28 08:26:36,1
51,thaiprong,รวมใจไทยปรองดอง,ทหารเป็นได้ทุกอย่าจริงๆ ไวรัสโคโรนาสายพันธุ์ให...,2020-01-28 08:19:02,1
52,trueman98587845,trueman,กระผมหนึ่งในพลเมืองคนไทย ขอเป็นกำลังใจให้คุณหม...,2020-01-31 05:47:36,1
53,trueman98587845,trueman,ต้องบอกว่ารัฐพยายามประชาสัมพันธ์ให้ผู้คนเกิดคว...,2020-01-31 04:10:54,1
54,trueman98587845,trueman,กระผมหนึ่งในพลเมืองคนไทย ขอเป็นกำลังใจให้คุณหม...,2020-01-31 03:46:25,1
55,trueman98587845,trueman,ต้องบอกว่ารัฐพยายามประชาสัมพันธ์ให้ผู้คนเกิดคว...,2020-01-31 03:28:00,1
56,trueman98587845,trueman,ต้องบอกว่ารัฐพยายามประชาสัมพันธ์ให้ผู้คนเกิดคว...,2020-01-31 03:27:51,1
57,trueman98587845,trueman,กระผมหนึ่งในพลเมืองคนไทย ขอเป็นกำลังใจให้คุณหม...,2020-01-31 03:26:24,1
58,trueman98587845,trueman,กระผมหนึ่งในพลเมืองคนไทย ขอเป็นกำลังใจให้คุณหม...,2020-01-31 03:22:46,1
59,trueman98587845,trueman,กระผมหนึ่งในพลเมืองคนไทย ขอเป็นกำลังใจให้คุณหม...,2020-01-31 03:22:40,1


In [0]:
io_df.iloc[[70, 94, 99, 100]][['screen_name', 'fullname', 'cleaned_text', 'timestamp', 'is_IO']]

,screen_name,fullname,cleaned_text,timestamp,is_IO
70,tawannak32,ทวารหนัก,ข่าวปลอมครับ ในเรื่องไวรัสโคโรนา มีข่าวปลอมจำน...,2020-01-29 12:37:52,1
94,tawannak32,ทวารหนัก,ข่าวปลอมครับ ใช้วิจารณญาณในการวิเคราะห์ด้วยน่ะ...,2020-01-30 13:34:49,1
99,sakonkob2,sakonkob,ข่าวปลอมครับ ใช้วิจารณญาณในการวิเคราะห์ด้วยน่ะ...,2020-01-30 13:33:12,1
100,sakonkob2,sakonkob,ข่าวปลอมครับ ใช้วิจารณญาณในการวิเคราะห์ด้วยน่ะ...,2020-01-30 13:32:51,1


## Similarities between posts and users

following this tutorial: https://dev.to/coderasha/compare-documents-similarity-using-python-nlp-4odp

In [0]:
# example post
io_df.iloc[70]['cleaned_text']

'ข่าวปลอมครับ ในเรื่องไวรัสโคโรนา มีข่าวปลอมจำนวนมาก ควรวิเคราะห์ก่อนจะตกลงใจชื่อข่าวนั้นๆ หรือควรอ่านข่าวจากแหล่งที่เชื่อถือได้ครับ ไวรัสโคโรนา ไวรัสอู่ฮั่น รัฐบาลเตรียมพร้อม เสพสื่ออย่างมีสติ อย่าตื่นตระหนก Fakenews ข่าวปลอม'

In [0]:
gs_tfidf = gensim.models.TfidfModel(corpus)
for doc in gs_tfidf[corpus]:
    print([[dictionary[id], np.around(freq, decimals=2)] for id, freq in doc])

[['ข่าว', 0.28], ['ครับ', 0.12], ['คิดเห็น', 0.41], ['ค่อย', 0.38], ['งึวาท', 0.41], ['ดี', 0.17], ['ดู', 0.24], ['นะ', 0.16], ['ปลอม', 0.17], ['มา', 0.14], ['อย่า', 0.08], ['เชื่อ', 0.2], ['แล้ว', 0.17], ['แสด', 0.41], ['ให้', 0.09]]
[['ข่าว', 0.22], ['ดี', 0.26], ['ปลอม', 0.13], ['อย่า', 0.06], ['48', 0.32], ['Fakenews', 0.17], ['ขึ้น', 0.16], ['ชั่วโมง', 0.32], ['ตื่นตระหนก', 0.07], ['ป่วย', 0.25], ['ผู้', 0.19], ['ภาย', 0.32], ['มี', 0.05], ['รักษา', 0.23], ['ราชวิถี', 0.32], ['สติ', 0.07], ['สื่อ', 0.07], ['อย่าง', 0.06], ['อาการ', 0.32], ['อู่ฮั่น', 0.07], ['เสพ', 0.07], ['แพทย์', 0.28], ['โคโรนา', 0.15], ['ใน', 0.09], ['ไวรัส', 0.13]]
[['ข่าว', 0.28], ['ครับ', 0.12], ['คิดเห็น', 0.41], ['ค่อย', 0.38], ['งึวาท', 0.41], ['ดี', 0.17], ['ดู', 0.24], ['นะ', 0.16], ['ปลอม', 0.17], ['มา', 0.14], ['อย่า', 0.08], ['เชื่อ', 0.2], ['แล้ว', 0.17], ['แสด', 0.41], ['ให้', 0.09]]
[['ข่าว', 0.22], ['ดี', 0.26], ['ปลอม', 0.13], ['อย่า', 0.06], ['48', 0.32], ['Fakenews', 0.17], ['ขึ้น', 0.16], ['

In [0]:
# follow this: https://radimrehurek.com/gensim/similarities/docsim.html
from gensim.test.utils import datapath, get_tmpfile
index_temp = get_tmpfile("index")
index = gensim.similarities.Similarity(index_temp, corpus, 
                                       num_features=len(dictionary))
query_doc = io_df.iloc[70]['reduced_tokens']
query_doc_bow = dictionary.doc2bow(query_doc)
query = gs_tfidf[query_doc_bow]
result = index[query]
result

array([0.32306066, 0.4083323 , 0.32306066, 0.4083323 , 0.3889449 ,
       0.1988669 , 0.11641971, 0.12287238, 0.3889449 , 0.1988669 ,
       0.11641971, 0.12287238, 0.07560226, 0.0681395 , 0.06569998,
       0.0568549 , 0.07249698, 0.04423049, 0.        , 0.09137265,
       0.03867844, 0.07560226, 0.0681395 , 0.06569998, 0.0568549 ,
       0.07249698, 0.04423049, 0.        , 0.09137265, 0.03867844,
       0.08509724, 0.19316351, 0.19316351, 0.17089422, 0.17089422,
       0.17089422, 0.15929593, 0.15929593, 0.2809665 , 0.14656898,
       0.20543657, 0.08509724, 0.19316351, 0.19316351, 0.17089422,
       0.17089422, 0.17089422, 0.15929593, 0.15929593, 0.2809665 ,
       0.14656898, 0.20543657, 0.11641971, 0.1988669 , 0.11641971,
       0.1648616 , 0.1648616 , 0.11641971, 0.11641971, 0.11641971,
       0.1988669 , 0.11641971, 0.1988669 , 0.11641971, 0.1648616 ,
       0.1648616 , 0.11641971, 0.11641971, 0.11641971, 0.1988669 ,
       0.8909088 , 0.1988669 , 0.1988669 , 0.38779017, 0.38779

In [0]:
res_indices = np.argwhere(result > 0.5).flatten()

In [0]:
io_df.iloc[list(res_indices),:]

,screen_name,fullname,text,timestamp,following,followers,num_tweets,location,num_likes,id,date_joined,num_days_joined,days_on_twitter,IO_meta_clf,isIO_meta_prob,is_IO,is_IO_prob,cleaned_text,tokenized_text,normalized_text,reduced_tokens,bigram_tokens,reduced_bitokens
70,tawannak32,ทวารหนัก,ข่าวปลอมครับ ในเรื่องไวรัสโคโรนา มีข่าวปลอมจำน...,2020-01-29 12:37:52,34,5,107,-1,0,1212927650873786368,2020-01-02 18:44:00,55 days 05:16:00.000000000,55,1,0.995602,1,0.947547,ข่าวปลอมครับ ในเรื่องไวรัสโคโรนา มีข่าวปลอมจำน...,"[ข่าว, ปลอม, ครับ, , ใน, เรื่อง, ไวรัส, โคโรน...","[ข่าว, ปลอม, ครับ, ใน, เรื่อง, ไวรัส, โคโรนา, ...","[ข่าว, ปลอม, ครับ, ใน, เรื่อง, ไวรัส, โคโรนา, ...","[ข่าว ปลอม, ปลอม ครับ, ครับ ใน, ใน เรื่อง, เรื...","[ข่าว ปลอม, ปลอม ครับ, ครับ ใน, ใน เรื่อง, เรื..."
82,tawannak32,ทวารหนัก,ข่าวปลอมครับ ในเรื่องไวรัสโคโรนา มีข่าวปลอมจำน...,2020-01-29 12:37:52,34,5,107,-1,0,1212927650873786368,2020-01-02 18:44:00,55 days 05:16:00.000000000,55,1,0.995602,1,0.947547,ข่าวปลอมครับ ในเรื่องไวรัสโคโรนา มีข่าวปลอมจำน...,"[ข่าว, ปลอม, ครับ, , ใน, เรื่อง, ไวรัส, โคโรน...","[ข่าว, ปลอม, ครับ, ใน, เรื่อง, ไวรัส, โคโรนา, ...","[ข่าว, ปลอม, ครับ, ใน, เรื่อง, ไวรัส, โคโรนา, ...","[ข่าว ปลอม, ปลอม ครับ, ครับ ใน, ใน เรื่อง, เรื...","[ข่าว ปลอม, ปลอม ครับ, ครับ ใน, ใน เรื่อง, เรื..."
94,tawannak32,ทวารหนัก,ข่าวปลอมครับ\nใช้วิจารณญาณในการวิเคราะห์ด้วยน่...,2020-01-30 13:34:49,34,5,107,-1,0,1212927650873786368,2020-01-02 18:44:00,55 days 05:16:00.000000000,55,1,0.995602,1,0.949400,ข่าวปลอมครับ ใช้วิจารณญาณในการวิเคราะห์ด้วยน่ะ...,"[ข่าว, ปลอม, ครับ, , ใช้, วิจารณญาณ, ใน, การ,...","[ข่าว, ปลอม, ครับ, ใช้, วิจารณญาณ, ใน, การ, วิ...","[ข่าว, ปลอม, ครับ, ใช้, วิจารณญาณ, ใน, การ, วิ...","[ข่าว ปลอม, ปลอม ครับ, ครับ ใช้, ใช้ วิจารณญาณ...","[ข่าว ปลอม, ปลอม ครับ, ครับ ใช้, ใช้ วิจารณญาณ..."
95,tawannak32,ทวารหนัก,ข่าวปลอมครับ\nใช้วิจารณญาณในการวิเคราะห์ด้วยน่...,2020-01-30 13:34:40,34,5,107,-1,0,1212927650873786368,2020-01-02 18:44:00,55 days 05:16:00.000000000,55,1,0.995602,1,0.949400,ข่าวปลอมครับ ใช้วิจารณญาณในการวิเคราะห์ด้วยน่ะ...,"[ข่าว, ปลอม, ครับ, , ใช้, วิจารณญาณ, ใน, การ,...","[ข่าว, ปลอม, ครับ, ใช้, วิจารณญาณ, ใน, การ, วิ...","[ข่าว, ปลอม, ครับ, ใช้, วิจารณญาณ, ใน, การ, วิ...","[ข่าว ปลอม, ปลอม ครับ, ครับ ใช้, ใช้ วิจารณญาณ...","[ข่าว ปลอม, ปลอม ครับ, ครับ ใช้, ใช้ วิจารณญาณ..."
96,tawannak32,ทวารหนัก,ข่าวปลอมครับ\nใช้วิจารณญาณในการวิเคราะห์ด้วยน่...,2020-01-30 13:34:28,34,5,107,-1,1,1212927650873786368,2020-01-02 18:44:00,55 days 05:16:00.000000000,55,1,0.995602,1,0.949400,ข่าวปลอมครับ ใช้วิจารณญาณในการวิเคราะห์ด้วยน่ะ...,"[ข่าว, ปลอม, ครับ, , ใช้, วิจารณญาณ, ใน, การ,...","[ข่าว, ปลอม, ครับ, ใช้, วิจารณญาณ, ใน, การ, วิ...","[ข่าว, ปลอม, ครับ, ใช้, วิจารณญาณ, ใน, การ, วิ...","[ข่าว ปลอม, ปลอม ครับ, ครับ ใช้, ใช้ วิจารณญาณ...","[ข่าว ปลอม, ปลอม ครับ, ครับ ใช้, ใช้ วิจารณญาณ..."
97,tawannak32,ทวารหนัก,ข่าวปลอมครับ\nใช้วิจารณญาณในการวิเคราะห์ด้วยน่...,2020-01-30 13:34:17,34,5,107,-1,0,1212927650873786368,2020-01-02 18:44:00,55 days 05:16:00.000000000,55,1,0.995602,1,0.949400,ข่าวปลอมครับ ใช้วิจารณญาณในการวิเคราะห์ด้วยน่ะ...,"[ข่าว, ปลอม, ครับ, , ใช้, วิจารณญาณ, ใน, การ,...","[ข่าว, ปลอม, ครับ, ใช้, วิจารณญาณ, ใน, การ, วิ...","[ข่าว, ปลอม, ครับ, ใช้, วิจารณญาณ, ใน, การ, วิ...","[ข่าว ปลอม, ปลอม ครับ, ครับ ใช้, ใช้ วิจารณญาณ...","[ข่าว ปลอม, ปลอม ครับ, ครับ ใช้, ใช้ วิจารณญาณ..."
98,tawannak32,ทวารหนัก,ข่าวปลอมครับ\nใช้วิจารณญาณในการวิเคราะห์ด้วยน่...,2020-01-30 13:34:06,34,5,107,-1,0,1212927650873786368,2020-01-02 18:44:00,55 days 05:16:00.000000000,55,1,0.995602,1,0.949400,ข่าวปลอมครับ ใช้วิจารณญาณในการวิเคราะห์ด้วยน่ะ...,"[ข่าว, ปลอม, ครับ, , ใช้, วิจารณญาณ, ใน, การ,...","[ข่าว, ปลอม, ครับ, ใช้, วิจารณญาณ, ใน, การ, วิ...","[ข่าว, ปลอม, ครับ, ใช้, วิจารณญาณ, ใน, การ, วิ...","[ข่าว ปลอม, ปลอม ครับ, ครับ ใช้, ใช้ วิจารณญาณ...","[ข่าว ปลอม, ปลอม ครับ, ครับ ใช้, ใช้ วิจารณญาณ..."
99,sakonkob2,sakonkob,ข่าวปลอมครับ\nใช้วิจารณญาณในการวิเคราะห์ด้วยน่...,2020-01-30 13:33:12,35,2,28,-1,0,866978782556471296,2017-05-23 04:27:00,1009 days 19:33:00.000000000,100

In [0]:
io_df.describe().round(2)

,following,followers,num_tweets,num_likes,id,days_on_twitter,IO_meta_clf,isIO_meta_prob,is_IO,is_IO_prob
count,375.00,375.00,375.00,375.00,3.750000e+02,375.00,375.00,375.00,375.0,375.00
mean,106.69,8.71,709.43,2.69,1.202177e+18,74.77,1.00,0.94,1.0,0.90
std,102.21,18.34,774.64,23.84,1.185704e+17,239.21,0.05,0.11,0.0,0.11
min,2.00,0.00,1.00,0.00,4.515064e+08,13.00,0.00,0.42,1.0,0.35
25%,34.00,2.00,147.00,0.00,1.212928e+18,14.00,1.00,0.93,1.0,0.92
50%,53.00,4.00,493.00,0.00,1.220152e+18,35.00,1.00,0.99,1.0,0.95
75%,190.00,5.00,1032.00,1.00,1.227765e+18,55.00,1.00,1.00,1.0,0.95
max,811.00,133.00,4606.00,420.00,1.228032e+18,2979.00,1.00,1.00,1.0,0.98
